In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''!pip install transformers

!pip install tensorflow==2.8
!pip install tensorflow_probability==0.12.2
from transformers import pipeline'''

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load('en_core_web_lg')
import numpy as np
import pandas as pd
!pip install bert-extractive-summarizer
#from summarizer import Summarizer
from summarizer import Summarizer,TransformerSummarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 9.6 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 34.9 MB/s 
     |████████████████████████████████| 163 kB 74.5 MB/s 
     |████████████████████████████████| 7.6 MB 60.4 MB/s 


In [ ]:
# keep only documents with covid -cov-2 and cov2
def search_focus(df):
    dfa = df[df['abstract'].str.contains('covid')]
    dfb = df[df['abstract'].str.contains('-cov-2')]
    dfc = df[df['abstract'].str.contains('cov2')]
    dfd = df[df['abstract'].str.contains('ncov')]
    dfe = df[df['abstract'].str.contains('vaccine')]
    frames=[dfa,dfb,dfc,dfd,dfe]
    df = pd.concat(frames)
    df=df.drop_duplicates(subset='title', keep="first")
    return df

# load the meta data from the CSV file using 3 columns (abstract, title, authors),
df=pd.read_excel('/content/drive/MyDrive/metadataFinal.xlsx', usecols=['title','journal','abstract','authors','doi','publish_time','sha'])

print (df.shape)

#fill na fields
df=df.fillna('no data provided')

#drop duplicate titles
df = df.drop_duplicates(subset='title', keep="first")

#keep only 2020 dated papers
#df=df[df['publish_time'].str.contains('2020')]

# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()+df["title"].str.lower()

#show 5 lines of the new dataframe
df=search_focus(df)
print (df.shape)
df.head()

(463005, 7)
(275107, 7)


,sha,title,doi,abstract,publish_time,authors,journal
10,2ecd2df3b19e54d13d8877422fca3328f0fc5256,COVID-19 Vaccine Priority for People With Neur...,10.14740/jnr665,no data providedcovid-19 vaccine priority for ...,45402,"Pfeffer, Gerald; Jacob, Sarah; Preston, Jeffrey",J Neurol Res
13,746889ad718cbc5154785a86863262e1d31c71c1,"A ‘Multiomic’ Approach of Saliva Metabolomics,...",10.1016/j.gastha.2021.12.006,background and aims the sars-cov-2 pandemic ha...,44926,"Pozzi, Chiara; Levi, Riccardo; Braga, Daniele;...",Gastro Hep Advances
14,c42b3698a5925cca3f5d59df41aaaeabe84627c2,Chapter 8 Infectious Disease Emergencies,10.1016/b978-0-323-83375-2.00008-5,covid-19 has added new relevance to the relati...,44926,"Aliyeva, Gulnara Davud",Rapid Response Situations
15,d64935bc0335df545954b34b3bbff8ca249dfd73,"Two viruses, one prescription: slow down",10.1016/j.trpro.2021.12.034,the covid-19 pandemic has devastated communiti...,44926,"Tolley, Rodney; Tranter, Paul",Transportation Research Procedia
16,0268b9f3837b8e568b3ec59c69b285b41d725d69,A Workspace Typology for Enterprise Collaborat...,10.1016/j.procs.2021.12.017,the global covid-19 pandemic and the need for ...,44926,"Bahles, Sebastian; Schwade, Florian; Schubert,...",Procedia Computer Science


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from IPython.core.display import display, HTML

def remove_stopwords(text,stopwords):
    text = "".join(c for c in text if c not in ('!','.',',','?','(',')','-'))
    text_tokens = word_tokenize(text)
    
    #remove stopwords
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    str1=''
    str1=' '.join(word for word in tokens_without_sw)
    return str1


# Search - Questions we formulated; Sentence - Summarized answer
def score_sentence(search,sentence):
        main_doc=nlp(sentence)
        search_doc=nlp(search)
        ### DOCUMENT SIMILARITY WHICH IS BEING CALLED AS RELEVENCY SCORE 
        sent_score=main_doc.similarity(search_doc)
        return sent_score

def process_question(df,search,focus):
    df_table = pd.DataFrame(columns = ["pub_date","title","excerpt","rel_score"])
    df1 = df[df['abstract'].str.contains(focus)]
    search=remove_stopwords(search,stopwords)
    for index, row in df1.iterrows():
        sentences = row['abstract'].split('. ')
        pub_sentence=''
        hi_score=0
        for sentence in sentences:
            if len(sentence)>75:
                rel_score=score_sentence(search,sentence)
                if rel_score>.82:
                    sentence=sentence.capitalize()
                    if sentence[len(sentence)-1]!='.':
                        sentence=sentence+'.'
                    pub_sentence=pub_sentence+' '+sentence
                    if rel_score>hi_score:
                        hi_score=rel_score
        if pub_sentence!='':
            authors=row["authors"].split(" ")
            link=row['doi']
            title=row["title"]
            score=hi_score
            linka='https://doi.org/'+link
            linkb=title
            final_link='<p align="left"><a href="{}">{}</a></p>'.format(linka,linkb)
            #author_link='<p align="left"><a href="{}">{}</a></p>'.format(linka,authors[0]+' et al.')
            #sentence=pub_sentence+' '+author_link
            sentence=pub_sentence
            #sentence='<p fontsize=tiny" align="left">'+sentence+'</p>'
            to_append = [row['publish_time'],final_link,sentence,score]
            df_length = len(df_table)
            df_table.loc[df_length] = to_append
    df_table=df_table.sort_values(by=['rel_score'], ascending=False)
    return df_table

def prepare_summary_answer(text,model):
    #model = pipeline(task="summarization")
    return model(text)

###### MAIN PROGRAM ######
#model = Summarizer()
model = Summarizer('distilbert-base-uncased', hidden=[-1,-2], hidden_concat=True)
#model=TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
#model = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")
#GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")


# QUESTIONS

search=[
'What is the effectiveness of drugs being developed and tried to treat COVID-19 patients?',
'Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen clarithromycin, and minocyclinethat that may exert effects on viral replication',
'How are potential complications of Antibody-Dependent Enhancement ADE in vaccine recipients being researched?',
'Exploration of use of best animal models and their predictive value for a human vaccine',
'Capabilities to discover a therapeutic not vaccine for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.',
'Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up and identifying approaches for expanding production capacity to ensure equitable and timely distribution to populations in need',
'What research and work is being done to develop a universal vaccine for coronavirus',
'What work and research has been done to develop animal models and standardize challenge studies',
'What work and research has been done to develop prophylaxis clinical studies and prioritize in healthcare workers',
'Approaches to evaluate risk for enhanced disease after vaccination',
'Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models in conjunction with therapeutics'
]

# MAIN FOCUS KEYWORDS

focus=['drugs','drugs','antibodies','animal model','therapeutic','models','vaccine','model','drugs','vaccine','animal']

z=0

for question in search:
    # process with spacy model and return df
    df_table=process_question(df,question,focus[z])
    
    #use only the top 20 papers excerpts for summarizaiton
    df_answers=df_table.head(10)
    
    text=''
    
    #loop through df to assemble sentences
    for index, row in df_answers.iterrows():
        text=text+' '+row['excerpt']
    
    display(HTML('<h2>'+question+'</h2>'))
    
    #summarize questions
    summary_answer=prepare_summary_answer(text,model)
    
    #summary_answer=summary_answer[0]['summary_text']
    display(HTML('<h4> Summarized Answer: </h4><i>'+summary_answer+'</i>'))
    display(HTML('<h5>results limited to 5 for ease of scanning</h5>'))
    
    #limit the size of the df for the html table
    df_table=df_table.head(5)
    
    #convert df_table to html and display
    df_table=HTML(df_table.to_html(escape=False,index=False))
    display(df_table)
    
    z=z+1

print ('done') 

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

pub_date,title,excerpt,rel_score
44682,Development of humanized ACE2 mouse and rat models for COVID-19 research.,"Small animal models that recapitulate human covid-19 disease are urgently needed for better understanding the transmission and therapeutic measurement. In summary, these data suggest that our humanized ace2 models can be valuable for covid-19 research.development of humanized ace2 mouse and rat models for covid-19 research.",0.865095
43986,COVID-19 preclinical models: human angiotensin-converting enzyme 2 transgenic mice,Animal models that mimic the human disease are highly essential to develop therapeutics and vaccines against covid-19. This literature review aims to present the importance of utilizing the human ace2 transgenic mouse model to better understand the pathogenesis of covid-19 and develop both therapeutics and vaccines.covid-19 preclinical models: human angiotensin-converting enzyme 2 transgenic mice.,0.857399
44072,Coformulation with tattoo ink for immunological assessment of vaccine immunogenicity in the draining lymph node,"Non-human primates are a key pre-clinical animal model for human vaccine development, allowing both lymph node and circulating immune responses to be longitudinally sampled for correlates of vaccine efficacy.",0.854113
43945,Recombinant Live Attenuated Influenza Vaccine Viruses Carrying Conserved T Cell Epitopes of Human Adenoviruses Induce Functional Cytotoxic T Cell Responses and Protect Mice against both Infections,Our findings suggest that the developed recombinant vaccines can be used for combined protection against influenza and human adenoviruses and warrant further evaluation on humanized animal models and subsequent human trials.recombinant live attenuated influenza vaccine viruses carrying conserved t cell epitopes of human adenoviruses induce functional cytotoxic t cell responses and protect mice against both infections.,0.853110
44602,Evidence based dosing of convalescent plasma for COVID-19 in future trials,"Sources: as no dose finding human trials were ever performed, we reviewed covid-19 animal model studies and human trials that provide (in)direct data on the pharmacokinetics and pharmacodynamics of convp.",0.849497


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


pub_date,title,excerpt,rel_score
44007,Human and novel coronavirus infections in children: a review.,Clinical trials for therapeutics and vaccine development should include paediatric considerations.,0.919539
44153,Severe acute respiratory syndrome coronavirus‐2 natural animal reservoirs and experimental models: systematic review,"The role that animal hosts play in disease transmission is still understudied and researchers wish to find suitable animal models for fundamental research and drug discovery. In addition, virus replication in mice, mink, hamsters and ferrets resembled subclinical human infection, so these animals might serve as useful models for future studies to evaluate vaccines or antiviral agents and to study host‐pathogen interactions. Our review comprehensively summarized current evidence on sars‐cov‐2 infection in animals and their usefulness as models for studying vaccines and antiviral drugs. Our findings may direct future studies for vaccine development, antiviral drugs and therapeutic agents to manage sars‐cov‐2‐caused diseases.severe acute respiratory syndrome coronavirus‐2 natural animal reservoirs and experimental models: systematic review.",0.915009
44565,Emerging vaccine nanotechnology: From defense against infection to sniping cancer,"Beyond infectious diseases, vaccine nanotechnology also exhibits considerable potential for cancer treatment. In this review, we summarize the recent advances in vaccine nanotechnology from infectious disease prevention to cancer immunotherapy and highlight the different types of materials, mechanisms, administration methods, as well as future perspectives.emerging vaccine nanotechnology: from defense against infection to sniping cancer.",0.909217
44697,"Cancer vaccines: past, present and future; a review article","Immunotherapy and vaccines have revolutionized disease treatment and prevention. These include preventative vaccines against infectious agents associated with cancers, and therapeutic vaccines used as immunotherapy agents to treat cancers. Challenges in developing cancer vaccines include heterogeneity within and between cancer types, screening and identification of appropriate tumour-specific antigens, and the choice of vaccine delivery platforms. Further advances in these areas are expected to facilitate development of effective novel cancer vaccines.",0.906512
44389,Therapeutic antibodies under development for SARS‐CoV‐2,Several antibody drugs have successfully entered clinical trials and achieved impressive therapeutic effects.,0.906153


pub_date,title,excerpt,rel_score
44246,A Bayesian Mixture Model for Predicting the COVID-19 Related Mortality in the United States,"We propose a bayesian mixture model to predict and monitor covid-19 mortality across the united states. Reliable models for predicting the covid-19 pandemic are essential to informing resource allocation and intervention strategies. A bayesian mixture model was able to more accurately predict the shape of the mortality curves across the united states than other models, including the timing of multiple peaks. However, given the dynamic nature of the pandemic, it is important that the results be updated regularly to identify and better monitor future waves, and characterize the epidemiology of the pandemic.a bayesian mixture model for predicting the covid-19 related mortality in the united states.",0.934710
44117,Dynamics models for identifying the key transmission parameters of the COVID-19 disease,"There are around 100 research groups across the world trying to develop a vaccine for this coronavirus. Therefore, the quantitative and qualitative analysis of the covid–19 pandemic is needed along with the effect of rapid test infection identification on controlling the spread of covid-19. Mathematical models with computational simulations are the effective tools that help global efforts to estimate key transmission parameters and further improvements for controlling this disease. In this paper, we develop the models for coronavirus disease at different stages with the addition of more parameters due to interactions among the individuals. Then, some key computational simulations and sensitivity analysis are investigated. Further, the local sensitivities for each model state concerning the model parameters are computed using the model reduction techniques: the dynamical models are eventually changed with the change of parameters are represented graphically.dynamics models for identifying the key transmission parameters of the covid-19 disease.",0.934483
43976,A Quantitative and Qualitative Analysis of the COVID–19 Pandemic Model,"Global efforts around the world are focused on to discuss several health care strategies for minimizing the impact of the new coronavirus (covid-19) on the community. Mathematical models with computational simulations are effective tools that help global efforts to estimate key transmission parameters and further improvements for controlling this disease. This work reviews and develops some suggested models for the covid-19 that can address important questions about global health care and suggest important notes. Then, we suggest an updated model that includes a system of differential equations with transmission parameters. Some key computational simulations and sensitivity analysis are investigated. Also, the local sensitivities for each model state concerning the model parameters are computed using three different techniques: non-normalizations, half normalizations, and full normalizations. Results based on the computational simulations show that the model dynamics are significantly changed for different key model parameters. Interestingly, we identify that transition rates between asymptomatic infected with both reported and unreported symptomatic infected individuals are very sensitive parameters concerning model variables in spreading this disease. This helps international efforts to reduce the number of infected individuals from the disease and to prevent the propagation of new coronavirus more widely on the community. Another novelty of this paper is the identification of the critical model parameters, which makes it easy to be used by biologists with less knowledge of mathematical modeling and also facilitates the improvement of the model for future development theoretically and practically.a quantitative and qualitative analysis of the covid–19 pandemic model.",0.933320
44473,County-Level Planning for Efficient Distribution of Emergency Medical Countermeasures with Rea

KeyboardInterrupt: ignored